In [1]:
%load_ext autoreload
%autoreload 2
from Naver_Blog_Crawl import *
from multiprocessing_on_dill import Pool, cpu_count
from functools import partial
# from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
import os
import lxml
from collections import defaultdict

In [2]:
keyword = ["키치", "kitsch", "모노", "컬러풀", "유니크", "미드센츄리", "클래식", "빈티지", "레트로", "모던", "심플", "럭셔리", "프리미엄", "프로페셔널", "고사양", "전문가", "미니", "소형", "네츄럴", '우드', "화이트"]
eow = ["전자 제품", "가전 제품"]

search_keywords = []
for k in keyword:
    for e in eow:
        search_keywords.append(k + " " + e)


In [3]:
num_cores = 40
search_keywords_split = np.array_split(search_keywords, num_cores)

In [4]:
def collect_blog(keywords):
    naver_blog_crawl = NaverBlogCrawl()
    return naver_blog_crawl.collect_blog(keywords, keywords_start_idx = 0, maximum = 100)

In [ ]:
with Pool(num_cores) as p:
    results_async=p.map_async(collect_blog, search_keywords_split)
    results_async.get()

In [22]:
naver_blog_crawl = NaverBlogCrawl()
result = naver_blog_crawl.collect_blog(search_keywords[2], keywords_start_idx = 0, num_blogs= 1)
# pd.DataFrame(result).to_excel("temp.xlsx")

0, keyword : 모노 가전 제품, # of blogs : 1
{'title': '세종 키즈풀 &lt;<b>모노</b>빌&gt; 아이와 함께 실내 온수풀장에서...', 'link': 'https://blog.naver.com/bom_ann/222939562203', 'description': '식탁위에는 <b>가전제품</b> 사용법 및 온수 추가하는 방법, 주의사항 등의 안내사항이 꼼꼼하게 메모되어... 특히, <b>모노</b>빌의 샤워용품, 스킨케어<b>제품</b>은 모두 저자극 비건<b>제품</b>이라서 아이들에게도 마음편히 사용 할...', 'bloggername': '봄앤 기록장✍', 'bloggerlink': 'blog.naver.com/bom_ann', 'postdate': '20221127'}


In [ ]:
naver_blog_crawl = NaverBlogCrawl()
result = naver_blog_crawl.collect_blog(search_keywords[:4], keywords_start_idx = 0, num_blogs= 1)
pd.DataFrame(result).to_excel("temp.xlsx")

In [ ]:
naver_blog_crawl = NaverBlogCrawl() # 23.8
result = naver_blog_crawl.collect_blog("LG스탠바이미")

In [6]:
def merge_dict(org_dict:dict, new_dict:dict, type:str = "full"):
    if type == "full":
        keys = pd.unique(list(org_dict.keys()) + list(new_dict.keys()))
    elif type == "left":
        keys = list(org_dict.keys())
    elif type == "right":
        keys = list(new_dict.keys())


    for key in keys:
        if isinstance(new_dict[key], list):
            org_dict[key].extend(new_dict[key])
        else:
            org_dict[key].append(new_dict[key])
    
    return org_dict

In [7]:
result_dict = defaultdict(list)
blog_dicts = os.listdir("./blog_dict")
for x in blog_dicts:
    with open(f"./blog_dict/{x}", "rb") as f:
        blog_dict = pickle.load(f)
        result_dict = merge_dict(result_dict, blog_dict)

In [9]:
result_df = pd.DataFrame(result_dict).drop_duplicates("blog_url")

In [ ]:
result_df["keyword"].drop_duplicates()

In [12]:
duplicated_blogs = result_df[["writer_name", "post_date"]].value_counts()
duplicated_blogs = duplicated_blogs.loc[duplicated_blogs>1].reset_index()

In [13]:
result_df = result_df.loc[result_df["writer_name"].isin(duplicated_blogs["writer_name"])==False]

In [9]:
def save_images(img_urls:str, img_names:str, img_dir:str = "./image"):
    
    if not os.path.exists(img_dir):
        os.makedirs(img_dir)


    for img_url, img_name in zip(img_urls, img_names):
        # image 저장        
        try:
            img_rq = requests.get(img_url, verify = False)
            with open(f"{img_dir}/{img_name.replace('/','_')}", "wb") as f:
                f.write(img_rq.content)
        except: 
            print(f"img_name : {img_name}, img_url : {img_url}")

In [241]:
from multiprocessing_on_dill import Pool, cpu_count
from functools import partial
# from tqdm import tqdm
import pandas as pd
import numpy as np


def parallelize_dataframe(df, 
                          func, 
                          group_keys = None, 
                          num_cores = None,
                          **params):
    """
        Pandas DataFrame의 apply함수를 병렬처리 하는 함수
        Args: 
            df: 적용 대상 데이터프레임 (Pandas.DataFrame)
            group_keys: apply를 적용할 때 기준이되는 group by key
        Returns:
            df: (Pandas.DataFrame)
            
        Exception: 
    """

    if num_cores is None:
        num_cores = cpu_count()

    if group_keys is None:
        df_list = np.array_split(df, num_cores)
    elif group_keys is not None:
        gr_df = df.groupby(group_keys)
        df_list = [group for name, group in gr_df]
    
#     tqdm.pandas()
    #func = partial(func, **params)
    
    # def map_func(data):
    #     return data.apply(func, axis = 1, **params)
#         return data.progress_apply(func, axis = 1, **params)
        
        
    with Pool(num_cores) as p:
        p.map(func, df_list)
#         pd_result = p.starmap(func, **params)

In [242]:
def save_images_par(temp_df):
    for row in temp_df[["keyword", "images", "image_urls"]].iterrows():
        save_images(img_urls = row[1][2], img_names = row[1][1], img_dir = f"/mnt/d/Crawl/Naver_Blog_Crawl/sample_image/{row[1][0].replace('/','_')}")

In [ ]:
parallelize_dataframe(df = result_df, func = save_images_par, group_keys = "keyword", num_cores = 40)

In [14]:
result_df = result_df.loc[result_df["writer_name"].str.contains("""(업체)|(시공)|(구매대행)|(시스템)|(전문)|(판매)|
                                                                   (센터)|(중고)|(청소)|(가전)|(렌탈)|(베스트샵)|(삼성)|
                                                                   (리폼)|(교체)|(홈쇼핑)|(닷컴)|(공식)|(\(주\))|(특가)|(이벤트)|(할인)|
                                                                   (신문)|(선택장애 살아가기)|(삼시육끼)""")==False]
result_df = result_df.loc[result_df["title"].str.contains("""(업체)|(시공)|(구매대행)|(시스템)|(전문)|(판매)|(센터)|
                                                             (중고)|(청소)|(가전)|(렌탈)|(베스트샵)|(삼성)|(리폼)|(교체)|
                                                             (체험단)|(홈쇼핑)|(닷컴)|(기부)|(방송)|(특가)|(이벤트)|(할인)|
                                                             (리스트)|(요약)|(리포트)""")==False]
result_df = result_df.loc[result_df["tags"].str.contains("""(업체)|(시공)|(구매대행)|(시스템)|(전문)|(판매)|(센터)|(중고)|(청소)|
                                                            (가전)|(렌탈)|(베스트샵)|(삼성)|(리폼)|(교체)|(체험단)|(홈쇼핑)|(닷컴)|
                                                            (특가)|(이벤트)|(할인)""")==False]

/tmp/ipykernel_13533/4030479231.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  result_df = result_df.loc[result_df["writer_name"].str.contains("""(업체)|(시공)|(구매대행)|(시스템)|(전문)|(판매)|
/tmp/ipykernel_13533/4030479231.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  result_df = result_df.loc[result_df["title"].str.contains("""(업체)|(시공)|(구매대행)|(시스템)|(전문)|(판매)|(센터)|
/tmp/ipykernel_13533/4030479231.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  result_df = result_df.loc[result_df["tags"].str.contains("""(업체)|(시공)|(구매대행)|(시스템)|(전문)|(판매)|(센터)|(중고)|(청소)|


In [15]:
result_df = result_df.loc[result_df["post_date"]>="2020000"]

In [16]:
result_df = result_df.loc[result_df["contents"].str.contains("""(랭킹)|(상세 페이지)|(체험단)|(입금)|(익일)|(검색)|(프로모션)|
                                                                (소식)|(포토툰)|(설치)|(닷컴)|(주소)|(전화)|(영업시간)|(스토어)|
                                                                ([\d]+원)|(010-)|(주식)|(카카오톡)|(채널)|(주가)|(주주)|(차트)|(전시)|
                                                                (가정의)|([\d]{3,4}-[\d]{3,4}-[\d]{4})|(이벤트)|(바로가기)|(도서)|(리스트)|
                                                                (요약)|(리포트)|(클릭)|(할\.인)|(구\.매)|(전문점)|(교환)|(소정)|(기자)|(출처)|
                                                                (소공미디어)|(정시)|(접수)|(참여)|(관련주)|(매수)""")==False]

/tmp/ipykernel_13533/2667990024.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  result_df = result_df.loc[result_df["contents"].str.contains("""(랭킹)|(상세 페이지)|(체험단)|(입금)|(익일)|(검색)|(프로모션)|


In [226]:
result_df = result_df.loc[(result_df["image_urls"].apply(lambda x: len(x)) != 0)]

In [227]:
result_df = result_df.loc[result_df["image_urls"].apply(lambda x: len(x)) < 30]

In [21]:
result_df.to_excel("blog_data_230111.xlsx", index = False, encoding = "CP949")

/home/seho/.local/lib/python3.10/site-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [91]:
blogId = re.search(r"blogId=([a-zA-Z0-9_-]+)", blog_url).group(1)
logNoList = re.search(r"logNo=([\d]+)", blog_url).group(1)

In [89]:
from multiprocessing_on_dill import Pool, cpu_count
from functools import partial
# from tqdm import tqdm
import pandas as pd
import numpy as np


def parallelize_dataframe(df, 
                          func, 
                          group_keys = None, 
                          num_cores = None,
                          **params):
    """
        Pandas DataFrame의 apply함수를 병렬처리 하는 함수
        Args: 
            df: 적용 대상 데이터프레임 (Pandas.DataFrame)
            group_keys: apply를 적용할 때 기준이되는 group by key
        Returns:
            df: (Pandas.DataFrame)
            
        Exception: 
    """

    if num_cores is None:
        num_cores = cpu_count()

    if group_keys is None:
        df_list = np.array_split(df, num_cores)
    elif group_keys is not None:
        gr_df = df.groupby(group_keys)
        df_list = [group for name, group in gr_df]
    
#     tqdm.pandas()
    #func = partial(func, **params)
    
    # def map_func(data):
    #     return data.apply(func, axis = 1, **params)
#         return data.progress_apply(func, axis = 1, **params)
        
        
    with Pool(num_cores) as p:
        pd_result = pd.concat(p.map(func, df_list))
#         pd_result = p.starmap(func, **params)
        
    return pd_result

In [90]:
def get_blog_like_count(blog_url):
    blogId = re.search(r"blogId=([a-zA-Z0-9_-]+)", blog_url).group(1)
    logNoList = re.search(r"logNo=([\d]+)", blog_url).group(1)

    like_base_url = "https://blog.like.naver.com/v1/search/contents"
    params_dict = {"suppress_response_codes" : "true",
                   "q" : f"BLOG[{blogId}_{logNoList}]"}

    blog_rq = requests.get(like_base_url, params_dict)
    
    if len(blog_rq.json()["contents"][0]["reactions"]) == 0:
        like_count = 0
    else:
        like_count = blog_rq.json()["contents"][0]["reactions"][0]["count"]
    return like_count


In [91]:
def get_like_par(url_list):
    like_list = []
    blog_url_list = []
    for i, blog_url in enumerate(url_list):
        blog_url_list.append(blog_url)
        like_list.append(get_blog_like_count(blog_url))

    return pd.DataFrame({"blog_url" : blog_url_list, "like" : like_list})

In [21]:
like_df = parallelize_dataframe(df = result_df["blog_url"], func = get_like_par, num_cores = 50)

In [191]:
result_df = pd.merge(result_df, like_df, on = "blog_url")
print(result_df.shape)

(18431, 11)


In [192]:
result_df = pd.merge(result_df, product_data[["keyword", "category1Name", "category2Name", "category3Name", "category4Name"]].drop_duplicates("keyword"), on = "keyword")
print(result_df.shape)

(18431, 15)


In [193]:
result_df = result_df.loc[result_df["category2Name"] != "인테리어소품"]

In [194]:
category3_df = pd.read_csv("소카테고리.csv", encoding = "CP949")

In [195]:
result_df = pd.merge(result_df, category3_df, left_on = "category3Name", right_on = "소카테고리")

In [196]:
virtual_user_df = pd.read_excel("~/Project_C/1차오픈용계정_0428.xlsx")

In [197]:
result_df["알파카테고리"].unique()

array(['요리/주방', '음향/사운드', '영상가전', '생활가전', '뷰티가전', '계절가전', '건강가전', '의류관리기',
       '위생가전'], dtype=object)

In [198]:
alpha_category = ["음향/사운드", "계절가전", "영상가전", "요리/주방", "생활가전", "뷰티가전", "위생가전", "의류관리기", "건강가전"]

In [199]:
category_num = (virtual_user_df[alpha_category].isna() == False).sum().to_dict()

In [200]:
category_num

{'음향/사운드': 20,
 '계절가전': 40,
 '영상가전': 35,
 '요리/주방': 50,
 '생활가전': 60,
 '뷰티가전': 25,
 '위생가전': 15,
 '의류관리기': 10,
 '건강가전': 10}

In [234]:
sample_result_df = result_df.groupby("알파카테고리").apply(lambda x: x.sort_values("like", ascending = False).head(category_num[x["알파카테고리"].to_list()[0]]*5)).reset_index(drop=True)
sample_result_df["id"] = [f"blog_{i+1}" for i in range(sample_result_df.shape[0])]

In [235]:
sample_result_df.shape

(1325, 18)

In [236]:
import random

In [244]:
virtual_user_df["blog_list"] = [[] for _ in range(virtual_user_df.shape[0])]

In [245]:
for category in alpha_category:
    category_user = virtual_user_df.loc[virtual_user_df[category] == "O"]
    # num_user = category_user.shape[0]
    user_ids = category_user["번호"]
    
    sampled_list = []
    for user_id in user_ids:
        category_blog_id = sample_result_df.loc[(sample_result_df["알파카테고리"] == category) & (sample_result_df["id"].isin(sampled_list) == False), "id"].to_list()
        temp_sampled_list = random.sample(category_blog_id, 5)
        sampled_list.extend(temp_sampled_list)

        temp = virtual_user_df.loc[virtual_user_df["번호"] == user_id, "blog_list"]
        temp[temp.index.item()].extend(temp_sampled_list)

In [246]:
virtual_user_df

,번호,이메일(@test.com),닉네임,비밀번호,음향/사운드,계절가전,영상가전,요리/주방,생활가전,뷰티가전,위생가전,의류관리기,건강가전,blog_list
0,1,code1@test.com,ellys.home,code1234!,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[blog_1249, blog_1242, blog_1272, blog_1263, b..."
1,2,code2@test.com,cosydaily,code1234!,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[blog_1190, blog_1187, blog_1260, blog_1218, b..."
2,3,code3@test.com,applely88,code1234!,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[blog_1235, blog_1203, blog_1258, blog_1196, b..."
3,4,code4@test.com,jshpink82,code1234!,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[blog_1216, blog_1257, blog_1214, blog_1230, b..."
4,5,code5@test.com,zooty7979,code1234!,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[blog_1177, blog_1253, blog_1191, blog_1239, b..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,146,code146@test.com,wyatt_home,code1234!,NaN,NaN,NaN,O,NaN,O,NaN,NaN,NaN,"[blog_930, blog_975, blog_1070, blog_1049, blo..."
146,147,code147@test.com,yellow_8,code1234!,NaN,NaN,NaN,O,NaN,O,NaN,NaN,NaN,"[blog_987, blog_1076, blog_967, blog_1087, blo..."
147,148,code148@test.com,y3ll0w,code1234!,NaN,NaN,NaN,O,NaN,O,NaN,NaN,NaN,"[blog_882, blog_891, blog_973, blog_1028, blog..."
148,149,code149@test.com,for_yellow,code1234!,NaN,NaN,NaN,O,NaN,O,NaN,NaN,NaN,"[blog_864, blog_871, blog_1040, blog_883, blog..."


In [247]:
virtual_user_df.to_excel("가상유저_블로그_id_220502.xlsx")

In [240]:
sample_result_df.to_excel("sample_naver_blog_220502.xlsx", index = False)

In [243]:
parallelize_dataframe(df = sample_result_df, func = save_images_par, group_keys = "keyword", num_cores = 60)

img_name : LG전자 43NANO75KPA_생활속의 이득이 되는 정보를 수집하는 푸름나무_20210621_img_9.jpg, img_url : https://thisiswhymymomsmiled.com/wp-content/uploads/2021/06/20210621_191034-300x18.png
img_name : 로보락 S7_은`s 기쁜하루_20220226_img_17.jpg, img_url : https://tble.kr/camimg.php?tcode=MjIyNjQzLW5pZF8yMzk4NzQyMS0yMzUxMDcz
img_name : 로보락 S7_Bling Blossom_20211217_img_10.jpg, img_url : https://tble.kr/camimg.php?tcode=MjA1MDQwLW5pZF85MzA0ODMwNC0yMDg2ODg4


In [17]:
pd.DataFrame(result_dict).to_parquet("blog_data_220312.parquet", index = False)

In [7]:
pd.DataFrame(result_dict).to_excel("blog_data_220501.xlsx", index = False, encoding = "CP949")

In [205]:
virtual_user_df.to_excel("가상유저_블로그_id_220502.xlsx")

In [177]:
virtual_user_df.loc[virtual_user_df["번호"] == user_id, "blog_list"].index == 1

array([ True])

In [ ]:
virtual_user_df.loc[virtual_user_df["번호"] == user_id, "blog_list"]

In [141]:
sample_result_df.loc[sample_result_df["알파카테고리"] == category, "id"]

300    blog_301
301    blog_302
302    blog_303
303    blog_304
304    blog_305
         ...   
535    blog_536
536    blog_537
537    blog_538
538    blog_539
539    blog_540
Name: id, Length: 240, dtype: object

In [118]:
category_user = virtual_user_df.loc[virtual_user_df["음향/사운드"] == "O"]
num_user = category_user.shape[0]